Purpose of this notebook is to retrieve the urls from the program subpages, for Question Generation for RAG Evaluation

This is the first notebook. Afterwards, please go to `group_program_subpages.ipynb`

After running the cells under `Setup`, ensure that a folder named `program-sub-pages` is created in this `notebooks` folder in order to store the created excels later on.

<hr>

### Setup

Enable capturing of output & load in kedro.ipython extension

In [3]:
%%capture
%load_ext kedro.ipython

Imports

In [1]:
import os
import pandas as pd

In [26]:
folder_path = 'program-sub-pages'
if not os.path.exists(folder_path):
    # If it doesn't exist, create the folder
    os.makedirs(folder_path)
    print(f"Folder created: {folder_path}")
else:
    print(f"Folder already exists: {folder_path}")

Folder already exists: program-sub-pages


<hr>

### Load Data

List all the datasets registered in the Kedro DataCatalog

In [4]:
# ruff: noqa: F821
print("Total datasets:", len(catalog.list()), "\nName of datasets:")
catalog.list() # list all avaliable datasets

Total datasets: 185 
Name of datasets:



[
    'all_contents',
    'missing_contents',
    'all_contents_standardized',
    'all_contents_added',
    'all_contents_extracted',
    'all_extracted_text',
    'all_contents_mapped',
    'merged_data',
    'google_analytics_data',
    'raw_word_counts',
    'log_word_counts',
    'flag_for_removal_by_type',
    'recipes_data',
    'filtered_data_with_keywords',
    'embeddings_data',
    'weighted_embeddings',
    'ground_truth_data',
    'first_level_pred_cluster',
    'first_level_metrics',
    'first_level_cluster_size',
    'first_level_clustered_nodes',
    'first_level_unclustered_nodes',
    'final_cluster_size',
    'final_predicted_cluster',
    'final_clustered_nodes',
    'final_unclustered_nodes',
    'final_metrics',
    'filtered_data_rag',
    'processed_data_rag',
    'json_data_rag',
    'parameters',
    'params:columns_to_keep',
    'params:columns_to_keep.cost-and-financing',
    'params:columns_to_keep.diseases-and-conditions',
    'params:columns_to_keep.hea

Retrieve <mark><b>merged_data</b></mark> dataset [Merged collection of Health Hub articles across different content categories]

In [5]:
# ruff: noqa: F821
merged_data_og = catalog.load("merged_data")

                    INFO     Loading data from merged_data (ParquetDataset)...                  ]8;id=470721;file://c:\Users\hinat\Desktop\healthhub-content-optimization\hhcontentopt-dev\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=388865;file://c:\Users\hinat\Desktop\healthhub-content-optimization\hhcontentopt-dev\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

Save all information of program sub pages into excel sheet [Before data cleaning/removal later on in this notebook][Optional]

In [7]:
# relevant_categories = ['program-sub-pages', 'programs']
# df_programSubPages_all = merged_data_og[merged_data_og["content_category"].isin(relevant_categories)]

# writer = pd.ExcelWriter(r"program-sub-pages\all_programSubPages.xlsx", engine='xlsxwriter')

# names = ['cleaned_program_subpages_url']
# dataframes = [df_programSubPages_all]

# for name, frame in zip(names, dataframes):
#    frame.to_excel(writer, sheet_name=name, index=False)

# writer.close()

<hr>

#### Initial Data Cleaning (with 'remove_type')

Remove the rows with value: "No Extracted Content", "NaN", "No relevant content and mainly links", "Table of Contents", "No HTML Tags"

In [9]:
merged_data_og['remove_type'].value_counts()


remove_type
Below Word Count                        48
Recipe                                  48
No HTML Tags                            43
No Extracted Content                    29
Duplicated Content                      18
Infographic                             14
NaN                                     11
Multilingual                             6
No relevant content and mainly links     6
Table of Contents                        4
Services Directory                       3
Duplicated URL                           3
Name: count, dtype: int64

In [7]:
merged_data_og.columns


Index(['id', 'content_name', 'title', 'article_category_names',
       'cover_image_url', 'full_url', 'full_url2', 'friendly_url',
       'category_description', 'content_body', 'keywords', 'feature_title',
       'pr_name', 'alternate_image_text', 'date_modified', 'number_of_views',
       'last_month_view_count', 'last_two_months_view', 'content_category',
       'to_remove', 'remove_type', 'has_table', 'has_image',
       'related_sections', 'extracted_tables', 'extracted_raw_html_tables',
       'extracted_links', 'extracted_headers', 'extracted_images',
       'extracted_content_body', 'l1_mappings', 'l2_mappings', 'page_views',
       'engagement_rate', 'bounce_rate', 'exit_rate', 'scroll_percentage',
       'percentage_total_views', 'cumulative_percentage_total_views'],
      dtype='object')

In [11]:
values_to_drop = ["No Extracted Content", "NaN", "No relevant content and mainly links", "Table of Contents", "No HTML Tags"]

# Drop rows where 'remove_type' is in the specified list
merged_data = merged_data_og[~merged_data_og['remove_type'].isin(values_to_drop)]
# Optionally, you can also drop NaN values if required
merged_data['content_category'].value_counts()


content_category
live-healthy-articles          1142
medications                     579
diseases-and-conditions         324
program-sub-pages               303
programs                         61
medical-care-and-facilities      58
cost-and-financing               24
health-statistics                15
support-group-and-others         14
Name: count, dtype: int64

<hr>

#### Data Understanding

Check information of merged_data:

- No. of rows: 2613
- No. of columns: 37 (after dropping "to_remove" & "remove_type)
- Domain: Health Hub Articles

In [12]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2520 entries, 0 to 2612
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 2520 non-null   int64  
 1   content_name                       2520 non-null   object 
 2   title                              2520 non-null   object 
 3   article_category_names             1374 non-null   object 
 4   cover_image_url                    1598 non-null   object 
 5   full_url                           2519 non-null   object 
 6   full_url2                          2520 non-null   object 
 7   friendly_url                       2520 non-null   object 
 8   category_description               2502 non-null   object 
 9   content_body                       2520 non-null   object 
 10  keywords                           1378 non-null   object 
 11  feature_title                      761 non-null    object 
 1

Check column names
- More information of columns found under readme --> dataset portion

In [13]:
merged_data.columns


Index(['id', 'content_name', 'title', 'article_category_names',
       'cover_image_url', 'full_url', 'full_url2', 'friendly_url',
       'category_description', 'content_body', 'keywords', 'feature_title',
       'pr_name', 'alternate_image_text', 'date_modified', 'number_of_views',
       'last_month_view_count', 'last_two_months_view', 'content_category',
       'to_remove', 'remove_type', 'has_table', 'has_image',
       'related_sections', 'extracted_tables', 'extracted_raw_html_tables',
       'extracted_links', 'extracted_headers', 'extracted_images',
       'extracted_content_body', 'l1_mappings', 'l2_mappings', 'page_views',
       'engagement_rate', 'bounce_rate', 'exit_rate', 'scroll_percentage',
       'percentage_total_views', 'cumulative_percentage_total_views'],
      dtype='object')

Check the counts for each content_category. Ensure that it's 350 for program-sub-pages

In [45]:
merged_data["to_remove"].value_counts()


to_remove
False    2380
True      140
Name: count, dtype: int64

<hr>

### Prep Dataset for relevant categorie(s): programs & program-sub-pages

In [15]:
relevant_categories = ["programs", "program-sub-pages"]

df_programsubpages = merged_data[merged_data["content_category"].isin(relevant_categories)]
display(df_programsubpages)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,extracted_content_body,l1_mappings,l2_mappings,page_views,engagement_rate,bounce_rate,exit_rate,scroll_percentage,percentage_total_views,cumulative_percentage_total_views
2160,1434688,Screen for Life - National Health Screening Pr...,Screen for Life - National Health Screening Pr...,"Body Care,Conditions and Illnesses,Exercise an...",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/programmes/Screen_for...,www.healthhub.sg/programmes/Screen_for_Life,Screen_for_Life,Subsidised health screening for Singapore Citi...,"b'<div class=""ExternalClassAF91117F1CED4D888CC...",...,We note that some users have experienced issue...,None,None,1069166,0.511571,0.488429,0.605897,0.310903,0.359128,0.359128
2161,1434612,"Live Well, Age Well Programme","Live Well, Age Well Programme",None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/programmes/AAP,www.healthhub.sg/programmes/AAP,AAP,Stay healthy and active with FREE activities n...,"b'<div class=""ExternalClassBF0514EBE8F24C4A9A5...",...,"By 7 February 2023, all users must perform a o...",None,None,396253,0.486995,0.513005,0.771712,0.334680,0.133100,0.684081
2162,1434660,The Healthy 365 App,The Healthy 365 App,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/programmes/healthyliving,www.healthhub.sg/programmes/healthyliving,healthyliving,Start moving towards your health goals one day...,"b'<div class=""ExternalClassA8054F927A104E438A5...",...,[https://go.gov.sg/useh365] [https://go.gov.s...,None,None,571167,0.456685,0.543315,0.681652,0.312005,0.191853,0.550981
2163,1434580,"Eat, Drink, Shop Healthy Challenge","Eat, Drink, Shop Healthy Challenge",None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/programmes/eat-drink-...,www.healthhub.sg/programmes/eat-drink-shop-hea...,eat-drink-shop-healthy-challenge,Enter the world of sure wins when you choose h...,"b'<div class=""hpb-container nutrition"" data-pa...",...,Menu [#clear]\n\nMenu [#clear]\n\nMenu [...,None,None,91000,0.749137,0.250863,0.771615,0.460604,0.030567,0.782880
2164,1434657,Great things start when you MOVE IT!,Great things start when you MOVE IT!,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/programmes/LetsMoveIt,www.healthhub.sg/programmes/LetsMoveIt,LetsMoveIt,Physical activity is important to maintain a h...,"b'<div class=""hpb-container"" data-page-id=""hom...",...,[https://go.gov.sg/h365-moveit]\n\nGet Moving ...,None,None,101407,0.649578,0.350422,0.604731,0.354376,0.034062,0.752314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,1435127,Types of diabetes | Diabetes Hub_types-of-diab...,Types of diabetes | Diabetes Hub,None,None,https://www.healthhub.sg/programmes/diabetes-h...,www.healthhub.sg/programmes/diabetes-hub-v2/ty...,types-of-diabetes,Did you know there's more than 2 types of diab...,"b'<div class=""ExternalClass17FC6DDA290D4A3AA72...",...,3 BEaTMS TO BEAT DIABETES [/programmes/diabete...,None,None,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2588,1435167,Hypoglycaemia | Diabetes Hub_hypoglycaemia_Level1,Hypoglycaemia | Diabetes Hub,None,None,https://www.healthhub.sg/programmes/diabetes-h...,www.healthhub.sg/programmes/diabetes-hub-v2/hy...,hypoglycaemia,Find out what causes your blood sugar level to...,"b'<div class=""ExternalClass990CCF03E6B64C28A16...",...,3 BES TO BEAT DIABETES [/programmes/diabetes-h...,None,None,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2589,1468676,sya-test-1,Program Sub level 1,None,None,https://www.healthhub.sg/programmes/1test/sya-...,www.healthhub.sg/programmes/1test/sya-test-1/,sya-test-1,test1,"b'<p><a href=""""><img alt="""" src="""" style=""heig...",...,HealthHub\nRelaxation-ExerciseHealthhub\nList ...,None,None,0,0.000000,0.000000,0.000000,0.000

Check the number of articles for program-sub-pages + cost-and-financing

In [17]:
len(df_programsubpages)

364

Check information of dataset for program-sub-pages

Important to ensure that urls are all non-null

In [18]:
df_programsubpages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 364 entries, 2160 to 2595
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 364 non-null    int64  
 1   content_name                       364 non-null    object 
 2   title                              364 non-null    object 
 3   article_category_names             24 non-null     object 
 4   cover_image_url                    48 non-null     object 
 5   full_url                           363 non-null    object 
 6   full_url2                          364 non-null    object 
 7   friendly_url                       364 non-null    object 
 8   category_description               355 non-null    object 
 9   content_body                       364 non-null    object 
 10  keywords                           48 non-null     object 
 11  feature_title                      16 non-null     object 


Example of urls for the content categories

In [19]:
print(df_programsubpages["full_url"].iloc[0])

https://www.healthhub.sg/programmes/Screen_for_Life


<hr>

Check unqiue content names/article name

In [20]:
unique_contentName = df_programsubpages.content_name.unique()
print(len(unique_contentName))
unique_contentName

363



array(['Screen for Life - National Health Screening Programme',
       'Live Well, Age Well Programme', 'The Healthy 365 App',
       'Eat, Drink, Shop Healthy Challenge',
       'Great things start when you MOVE IT!',
       'Healthier SG | Enrol via HealthHub', 'Parent Hub',
       'National Steps Challenge™',
       'Nutrition Facts And Information On Eating Healthy',
       'STAY ONE STEP AHEAD WITH VACCINATIONS',
       "3 Be's To Beat Diabetes | Diabetes Hub",
       'Diabetes Hub: Guide to Managing Diabetes',
       'Youth Preventive Dental Service', 'HPB Rewards Programme',
       'Let’s BEAT Diabetes', 'I Quit Programme', 'HIV and AIDS',
       'Children’s Health E-Services', 'MindSG', 'Korang OK?',
       'Sexually Transmitted Infections',
       'Misuse of Antibiotics puts you at risk.',
       'Healthy Workplace Ecosystem', 'Health Promoting Malls',
       'Stroke Hub', 'Get Hands-on With Health',
       'Stay Well to Stay Strong', 'Hi #JustCheckingIn',
       'Take the fi

<hr>

### Prep dataframe of url for export

Name of relevant dataframes:
- df_programsubpages

<mark><b>Columns to include for Question Generation from url:</b></mark>
1. id: article ID
2. content_name: name of article stored as metadata
3. title: article title [same as 2.]
3. article_category_names: categories that articles can belong to
4. full_url: article url
5. friendly_url: file path w reference from content category
6. category_description: brief summary of article
7. content_body: html element of entire article body
8. keywords: article keywords
9. feature_title: feature title of articles
10. content_category: content category that article matches on HH --> <mark><b>"program-sub-pages" + "cost-and-financing"</b></mark>
11. extracted_content_body: text ver of html content
12. remove_type

<hr>

#### Creation of 2 DataFrames for Program-sub-pages into Excel Workbook

1. df_programsubpages_cols: with all necessary columns
2. df_programsubpages_urls: with only identifier + url columns

In [46]:
# Check list of columns
merged_data.columns


Index(['id', 'content_name', 'title', 'article_category_names',
       'cover_image_url', 'full_url', 'full_url2', 'friendly_url',
       'category_description', 'content_body', 'keywords', 'feature_title',
       'pr_name', 'alternate_image_text', 'date_modified', 'number_of_views',
       'last_month_view_count', 'last_two_months_view', 'content_category',
       'to_remove', 'remove_type', 'has_table', 'has_image',
       'related_sections', 'extracted_tables', 'extracted_raw_html_tables',
       'extracted_links', 'extracted_headers', 'extracted_images',
       'extracted_content_body', 'l1_mappings', 'l2_mappings', 'page_views',
       'engagement_rate', 'bounce_rate', 'exit_rate', 'scroll_percentage',
       'percentage_total_views', 'cumulative_percentage_total_views'],
      dtype='object')

In [47]:
# Prep columns to be included
relevantAllCols = ['id', 'content_name', 'title', 'full_url', 'friendly_url', 'category_description', 'extracted_content_body', 'content_body', 'keywords', 'content_category', 'remove_type']
urlCols = ['id', 'title', 'full_url', 'extracted_content_body', 'content_category']

In [48]:
# Create the dataframes
df_programsubpages_cols = df_programsubpages[relevantAllCols]
df_programsubpages_url = df_programsubpages[urlCols]

In [49]:
df_programsubpages_url

,id,title,full_url,extracted_content_body,content_category
2160,1434688,Screen for Life - National Health Screening Pr...,https://www.healthhub.sg/programmes/Screen_for...,We note that some users have experienced issue...,programs
2161,1434612,"Live Well, Age Well Programme",https://www.healthhub.sg/programmes/AAP,"By 7 February 2023, all users must perform a o...",programs
2162,1434660,The Healthy 365 App,https://www.healthhub.sg/programmes/healthyliving,[https://go.gov.sg/useh365] [https://go.gov.s...,programs
2163,1434580,"Eat, Drink, Shop Healthy Challenge",https://www.healthhub.sg/programmes/eat-drink-...,Menu [#clear]\n\nMenu [#clear]\n\nMenu [...,programs
2164,1434657,Great things start when you MOVE IT!,https://www.healthhub.sg/programmes/LetsMoveIt,[https://go.gov.sg/h365-moveit]\n\nGet Moving ...,programs
...,...,...,...,...,...
2587,1435127,Types of diabetes | Diabetes Hub,https://www.healthhub.sg/programmes/diabetes-h...,3 BEaTMS TO BEAT DIABETES [/programmes/diabete...,program-sub-pages
2588,1435167,Hypoglycaemia | Diabetes Hub,https://www.healthhub.sg/programmes/diabetes-h...,3 BES TO BEAT DIABETES [/programmes/diabetes-h...,program-sub-pages
2589,1468676,Program Sub level 1,https://www.healthhub.sg/programmes/1test/sya-...,HealthHub\nRelaxation-ExerciseHealthhub\nList ...,program-sub-pages
2593,1435215,If you have coronary heart disease | Diabetes Hub,https://www.healthhub.sg/programmes/diabetes-h...,3 BES TO BEAT DIABETES [/programmes/diabetes-h...,program-sub-pages


##### Export to csv

In [25]:
writer = pd.ExcelWriter(r"program-sub-pages\cleaned_programSubpages.xlsx", engine='xlsxwriter')

names = ['cleaned_program_subpages_url', 'cleaned_program_subpages_all']
dataframes = [df_programsubpages_url, df_programsubpages_cols]

for name, frame in zip(names, dataframes):
   frame.to_excel(writer, sheet_name=name, index=False)

writer.close()